# HW1, COMS 4995_005, Deep Learning

## Max Possible Score 120 Points  (100 + 20 Extra Credits)
----

# Part1: (Basic Neural Nework) (70 Points)

### Part 1.1:

- Divide the training data into 80% training set and 20% validation set. 
- Implement the functions in the ipython notebook so that you can train your network. 
- Your code should take network structure, training data, hyperparameters and generate validation set accuracy.
- Use Relu activation for intermediate layers and use cross entropy loss after taking softmax on the output of the final layer.

If you get all the things mentioned above working - **60 Points**

### Part 1.2:

Test your model accuracy on test set. If it is more than **47%**, you will get an additional score of **10 points**

# Part 2: (Regularization) (30 Points)

### Part 2.1 (15 Points) :

Modify code to add L2 regularization. Report the validation accuracy.

You should get a validation and test accuracy of more than the one reported in Part-1

### Part 2.2 (15 Points):

You should get a validation and test accuracy crossing **50%**


# Extra Credit (20 Points)

Show your excitement on deep learning! Top **3 scorers** will get these **20 points**

(Hints) Boost your accuracy by trying out: 
- Dropout Regularization
- Batch Normalization
- Other optimizers like Adam
- Learning Rate Decay
- Data Augmentation 
- Different Initializations for weights like Xaviers etc.


------------

## Code Guidelines:

1. Write your code in **Python 3**.
2. **DONOT** import any other packages.
3. Click **https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz** -> download **cifar-10-python.tar.gz** -> extract as **cifar-10-python**
4. Ensure that **this ipython notebook** and **cifar-10-python** folder are in the same folder.


-------------------



## Submission Guidelines:

1. Run this ipython notebook once and submit this file. Ensure that the outputs are printed properly. We will first see the outputs, if there are no outputs, we may not run the notebook at all.
2. Training on the **test data** is considered cheating. If we find any clue of that happening, then we will disqualify the submission and it will be reported accordingly.
3. Each team member needs to separately submit the the file named uni.ipynb on courseworks.
------------

## Team Information

Team Member1 (Name,UNI): 

Team Member2 (Name, UNI):

In [2]:
import numpy as np
import pickle
import copy
%matplotlib inline
import matplotlib.pyplot as plt
# Do not import other packages
# ys3031 Simon Sun pz2232

In [3]:
class FullyConnectedNetwork(object):
    """
    Abstraction of a Fully Connected Network.
    Stores parameters, activations, cached values. 
    You can add more functions in this class, and also modify inputs and outputs of each function
    
    """
    def __init__(self, layer_dim):
        """
        layer_dim: List containing layer dimensions. 
        
        Code:
        Initialize weight and biases for each layer
        """
#         self.W = None
#         self.b = None

        np.random.seed(1)
        
        self.W = []
        self.b = []
        i = 0
        while i<(len(layer_dimensions)-1):
            self.W.append((np.random.random((layer_dim[i+1],layer_dim[i]))-0.5)*0.02)
            self.b.append(np.zeros((layer_dim[i+1],1)))
            i = i+1
        self.num_layers = len(layer_dimensions)-1
#         self.drop_prob = drop_prob
#         self.reg_lambda = reg_lambda
        self.training_mode = 0
    
    def feedforward(self,X):
        """
        Expected Functionality: 
        Returns output of the neural network for input X. Also returns cache, which contains outputs of 
        intermediate layers which would be useful during backprop.
        """       
        cache = {"d_activation":[None]*(len(self.W)), "r_activation":[None]*(len(self.b)), "dm":[None]*(len(self.W)-1)}
        cache["d_activation"][0] = X
        cache["r_activation"][0] = X
        j = 1
        while j<len(self.W):
#             print(self.W[j-1].shape, cache["d_activation"][j-1].shape)
            cache["r_activation"][j] = self.relu_forward(self.affineForward(self.W[j-1], cache["d_activation"][j-1], self.b[j-1]))
            cache["d_activation"][j] = cache["r_activation"][j]
            j = j+1
            
        At = self.affineForward(self.W[j-1],cache["d_activation"][j-1],self.b[j-1])
        return At, cache
    
    def loss_function(self, At, Y):
        """
        At is the output of the last layer, returned by feedforward.
        Y contains true labels for this batch.
        this function takes softmax the last layer's output and calculates loss.
        the gradient of loss with respect to the activations of the last layer are also returned by this function.
        """
        exp_At = np.exp(At)
        prob = exp_At/np.sum(exp_At, axis=0, keepdims=True)
        logprob = -np.log(prob[Y,range(At.shape[1])])
        cost = np.sum(logprob)/At.shape[1]
        # gradient of cost
        dAt = prob
        dAt[Y,range(At.shape[1])] -= 1
        dAt /= At.shape[1]
        return cost, dAt
    
    def train(self, X, Y, max_iters=5000, batch_size=100, learning_rate=0.01, lambd=0,validate_every=200):
        """
        X: (3072 dimensions, 50000 examples) (Cifar train data)
        Y: (1 dimension, 50000 examples)
        lambd: the hyperparameter corresponding to L2 regularization
        
        Divide X, Y into train(80%) and val(20%), during training do evaluation on val set
        after every validate_every iterations and in the end use the parameters corresponding to the best
        val set to test on the Cifar test set. Print the accuracy that is calculated on the val set during 
        training. Also print the final test accuracy. Ensure that these printed values can be seen in the .ipynb file you
        submit.
        
        Expected Functionality: 
        This function will call functions feedforward, backprop and update_params. 
        Also, evaluate on the validation set for tuning the hyperparameters.
        """
        x_train, x_test = X[:, :40000], X[:, 40000:]
        y_train, y_test = Y[:, :40000], Y[:, 40000:]
#         print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
#         print("hello world!")
        i = 0
        cost = 200
        while i < max_iters and cost > 0.1:
            # get minibatch
            x_t,y_t = self.get_batch(x_train,y_train,batch_size)
            y_t = y_t.astype(int)
#             forward prop
#             self.training_mode = 1 #for training mode
            At,cache = self.feedforward(x_t)
#             print(At.shape)
#             self.training_mode = 0 #for testing mode
#             compute loss
            cost,dAL = self.loss_function(At,y_t)
            # compute gradients
            gradients = self.backprop(dAct=dAL,loss=cost,cache=cache)
            # update weights and biases based on gradient
#             print("The info of gradients is", gradients['dweights'][0].shape)
#             print("The info of W is", self.W[0].shape)
            self.updateParameters(gradients,learning_rate)
            if i % validate_every == 0:
                # print cost, train and validation set accuracies
                # print ("iteration:%i" % (i))
#                 print ("cost:%.2f, train accuracy:%.2f, validation accuracy:%.2f" % (cost,np.mean(self.predict(X_tr) == y_tr),np.mean(self.predict(X_val) == y_val)))
                print("Step is:", i, "| Cost is:", cost)
                if i % 5000 == 0 and i != 0:
                    learning_rate /= 2
#                     batch_size = min(batch_size*2, 40000)
            i += 1
    
    def affineForward(self, W, A, b):
        """
        Expected Functionality:
        Forward pass for the affine layer.
        :param A: input matrix, shape (L, S), where L is the number of hidden units in the previous layer and S is
        the number of samples
        :returns: the affine product WA + b, along with the cache required for the backward pass
        """
        return np.dot(W, A)+b
        
    
    def affineBackward(self, dA_prev, cache):
        """
        Expected Functionality:
        Backward pass for the affine layer.
        dA_prev: gradient from the next layer.
        cache: cache returned in affineForward
        :returns dA: gradient on the input to this layer
                 dW: gradient on the weights
                 db: gradient on the bias
        """
        dA = np.dot(self.W[self.num_layers].T, dA_prev)
        dW = np.dot(cache["d_activation"][self.num_layers], dA_prev.T,)
        db = np.sum(dA_prev, axis=1, keepdims=True)
        return dA, dW, db
        
    def relu_forward(self, X):
        """
        Expected Functionality:
        Forward pass of relu activation
        """
        return np.maximum(0, X)
        
    def relu_backward(self, dx, cached_x):
        """
        Expected Functionality:
        backward pass for relu activation
        """
        dx[cached_x["r_activation"][self.num_layers] <= 0] = 0
        return dx
    
    def get_batch(self, X, Y, batch_size):
        """
        Expected Functionality: 
        given the full training data (X, Y), return batches for each iteration of forward and backward prop.
        """
        choices = np.random.choice(range(40000), batch_size, replace=False)
        x_batch = np.zeros((X.shape[0], batch_size))
        y_batch = np.zeros((Y.shape[0], batch_size))
        for j, choice in enumerate(choices):
            x_batch[:, j] = X[:, choice]
            y_batch[:, j] = Y[:, choice]      
        return x_batch, y_batch 
        
    def backprop(self, loss, cache, dAct):
        """
        Expected Functionality: 
        returns gradients for all parameters in the network.
        dAct is the gradient of loss with respect to the output of final layer of the network.
        """
        gradients = {"dweights":[None]*(len(self.W)),"dbiases":[None]*(len(self.b))}
        i = len(self.W)-1
        self.num_layers = i
        temp_dA, temp_dW, temp_db = self.affineBackward(dAct, cache)
        gradients["dweights"][i] = temp_dW
        gradients["dbiases"][i] = temp_db
        temp_dA_next = temp_dA
        temp_dA_next = self.relu_backward(temp_dA_next, cache)
        i -= 1
        while i>=0:
            self.num_layers = i
            temp_dA, temp_dW, temp_db = self.affineBackward(temp_dA_next, cache)
            gradients["dweights"][i] = temp_dW
            gradients["dbiases"][i] = temp_db
            temp_dA_next = temp_dA
            temp_dA_next = self.relu_backward(temp_dA_next, cache)      
            i=i-1
            
        return gradients
    
    def updateParameters(self, gradients, learning_rate):
        """
        Expected Functionality:
        use gradients returned by backprop to update the parameters.
        """
        i=0
        while i<len(self.W):
#             print(gradients["dweights"][i].shape)
            self.W[i] += -learning_rate*gradients["dweights"][i].T
            self.b[i] += -learning_rate*gradients["dbiases"][i]
            i = i+1
    
    def evaluate(self, X_test, Y_test):
        '''
        X: X_test (3472 dimensions, 10000 examples)
        Y: Y_test (1 dimension, 10000 examples)
        
        Expected Functionality: 
        print accuracy on test set
        '''
        print("evaluation started")
        
        total = 0
        correct = 0.0
        Y_test = Y_test.astype(int)
        i = 0
        while i < len(X_test[0]):
            curr_x = X_test[:, i:i+1]
#             print(curr_x.shape)
            y_predicted, _ = self.feedforward(curr_x)
#             print(y_predicted, len(Y_test), y_predicted.shape)
#             print(Y_test[0][i], len(Y_test), Y_test.shape)
            if Y_test[0][i] == y_predicted.argmax():
                correct += 1
            total += 1
            i += 1
            if i % 50 == 0:
                print("Step:", i, "Correctness:", correct/total)
            
        return correct / total
            
        

In [4]:
class Loader:
    
    def unpickle(self, file):
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        return dict
    
    def load_train_data(self):
        '''
        loads training data: 50,000 examples with 3072 features
        '''
        X_train = None
        Y_train = None
        for i in range(1, 6):
            pickleFile = self.unpickle('./datasets/data_batch_{}'.format(i))
            dataX = pickleFile[b'data']
            dataY = pickleFile[b'labels']
            if type(X_train) is np.ndarray:
                X_train = np.concatenate((X_train, dataX))
                Y_train = np.concatenate((Y_train, dataY))
            else:
                X_train = dataX
                Y_train = dataY

        Y_train = Y_train.reshape(Y_train.shape[0], 1)

        return X_train.T, Y_train.T

    def load_test_data(self):
        '''
        loads testing data: 10,000 examples with 3072 features
        '''
        X_test = None
        Y_test = None
        pickleFile = self.unpickle('./datasets/test_batch')
        dataX = pickleFile[b'data']
        dataY = pickleFile[b'labels']
        if type(X_test) is np.ndarray:
            X_test = np.concatenate((X_test, dataX))
            Y_test = np.concatenate((Y_test, dataY))
        else:
            X_test = np.array(dataX)
            Y_test = np.array(dataY)

        Y_test = Y_test.reshape(Y_test.shape[0], 1)

        return X_test.T, Y_test.T

In [5]:
X_train,Y_train = Loader().load_train_data()
X_test, Y_test = Loader().load_test_data()

print("X_Train: {} -> {} examples, {} features".format(X_train.shape, X_train.shape[1], X_train.shape[0]))
print("Y_Train: {} -> {} examples, {} features".format(Y_train.shape, Y_train.shape[1], Y_train.shape[0]))
print("X_Test: {} -> {} examples, {} features".format(X_test.shape, X_test.shape[1], X_test.shape[0]))
print("Y_Test: {} -> {} examples, {} features".format(Y_test.shape, Y_test.shape[1], Y_test.shape[0]))

print(X_test.shape)

X_Train: (3072, 50000) -> 50000 examples, 3072 features
Y_Train: (1, 50000) -> 50000 examples, 1 features
X_Test: (3072, 10000) -> 10000 examples, 3072 features
Y_Test: (1, 10000) -> 10000 examples, 1 features
(3072, 10000)


## Part 1

In [6]:
layer_dimensions = [3072,1000,100, 10]  # including the input and output layers  
# 3072 is the input feature size, 10 is the number of outputs in the final layer
FCN = FullyConnectedNetwork(layer_dimensions)
FCN.train(X_train, Y_train, max_iters=200000, batch_size=500, learning_rate=0.001, lambd=0,validate_every=50)
# lambd, the L2 regularization penalty hyperparamter will be 0 for this part
y_predicted = FCN.evaluate(X_test, Y_test)  # print accuracy on test set

Step is: 0 | Cost is: 2.33485919854
Step is: 50 | Cost is: 2.0584906501
Step is: 100 | Cost is: 1.89812605093
Step is: 150 | Cost is: 1.93692050768
Step is: 200 | Cost is: 1.92153690478
Step is: 250 | Cost is: 1.92028675693
Step is: 300 | Cost is: 1.80126334884
Step is: 350 | Cost is: 1.79512468734
Step is: 400 | Cost is: 1.77871571262
Step is: 450 | Cost is: 1.71591139512
Step is: 500 | Cost is: 1.78423389318
Step is: 550 | Cost is: 1.71608791272
Step is: 600 | Cost is: 1.71751223619
Step is: 650 | Cost is: 1.72294601457
Step is: 700 | Cost is: 1.64875065919
Step is: 750 | Cost is: 1.71832594193
Step is: 800 | Cost is: 1.63279952278
Step is: 850 | Cost is: 1.65941489622
Step is: 900 | Cost is: 1.5995758677
Step is: 950 | Cost is: 1.61471408095
Step is: 1000 | Cost is: 1.58398830019
Step is: 1050 | Cost is: 1.53720964692
Step is: 1100 | Cost is: 1.58850396271
Step is: 1150 | Cost is: 1.58037352251
Step is: 1200 | Cost is: 1.50241474099
Step is: 1250 | Cost is: 1.54238896025
Step is: 13

Step is: 10500 | Cost is: 0.780824610352
Step is: 10550 | Cost is: 0.71674439633
Step is: 10600 | Cost is: 0.773301536277
Step is: 10650 | Cost is: 0.708205387731
Step is: 10700 | Cost is: 0.706832939079
Step is: 10750 | Cost is: 0.773080239894
Step is: 10800 | Cost is: 0.706353238997
Step is: 10850 | Cost is: 0.764409133477
Step is: 10900 | Cost is: 0.799593664714
Step is: 10950 | Cost is: 0.72178373591
Step is: 11000 | Cost is: 0.71872079571
Step is: 11050 | Cost is: 0.713386312977
Step is: 11100 | Cost is: 0.770383237885
Step is: 11150 | Cost is: 0.738363547477
Step is: 11200 | Cost is: 0.681059701271
Step is: 11250 | Cost is: 0.750952691045
Step is: 11300 | Cost is: 0.701694829377
Step is: 11350 | Cost is: 0.669641384027
Step is: 11400 | Cost is: 0.678730743239
Step is: 11450 | Cost is: 0.763591218544
Step is: 11500 | Cost is: 0.69471987494
Step is: 11550 | Cost is: 0.71586829508
Step is: 11600 | Cost is: 0.689009083998
Step is: 11650 | Cost is: 0.671472767667
Step is: 11700 | Cost

Step is: 20550 | Cost is: 0.460414471632
Step is: 20600 | Cost is: 0.469072207244
Step is: 20650 | Cost is: 0.446861351396
Step is: 20700 | Cost is: 0.482960032515
Step is: 20750 | Cost is: 0.431823630288
Step is: 20800 | Cost is: 0.446650535712
Step is: 20850 | Cost is: 0.438857112714
Step is: 20900 | Cost is: 0.431826450185
Step is: 20950 | Cost is: 0.485793704843
Step is: 21000 | Cost is: 0.433237509626
Step is: 21050 | Cost is: 0.458104031631
Step is: 21100 | Cost is: 0.463347098426
Step is: 21150 | Cost is: 0.495712767067
Step is: 21200 | Cost is: 0.468837936239
Step is: 21250 | Cost is: 0.418447479607
Step is: 21300 | Cost is: 0.444517832893
Step is: 21350 | Cost is: 0.411270173197
Step is: 21400 | Cost is: 0.471864503672
Step is: 21450 | Cost is: 0.471779213789
Step is: 21500 | Cost is: 0.455747140046
Step is: 21550 | Cost is: 0.428278149819
Step is: 21600 | Cost is: 0.406581559485
Step is: 21650 | Cost is: 0.425798939888
Step is: 21700 | Cost is: 0.411009584493
Step is: 21750 |

Step is: 30600 | Cost is: 0.396154597019
Step is: 30650 | Cost is: 0.388116427689
Step is: 30700 | Cost is: 0.440763408286
Step is: 30750 | Cost is: 0.349453775544
Step is: 30800 | Cost is: 0.387487067278
Step is: 30850 | Cost is: 0.453598467998
Step is: 30900 | Cost is: 0.37411255304
Step is: 30950 | Cost is: 0.432498241285
Step is: 31000 | Cost is: 0.388313919248
Step is: 31050 | Cost is: 0.396562105155
Step is: 31100 | Cost is: 0.314803274578
Step is: 31150 | Cost is: 0.398538207134
Step is: 31200 | Cost is: 0.366897628108
Step is: 31250 | Cost is: 0.365034467293
Step is: 31300 | Cost is: 0.359751605209
Step is: 31350 | Cost is: 0.413945222884
Step is: 31400 | Cost is: 0.377654993686
Step is: 31450 | Cost is: 0.383121296059
Step is: 31500 | Cost is: 0.381727997634
Step is: 31550 | Cost is: 0.410297056063
Step is: 31600 | Cost is: 0.407418367094
Step is: 31650 | Cost is: 0.385050281883
Step is: 31700 | Cost is: 0.37392815119
Step is: 31750 | Cost is: 0.356282145089
Step is: 31800 | C

Step is: 40650 | Cost is: 0.39686727774
Step is: 40700 | Cost is: 0.361039098723
Step is: 40750 | Cost is: 0.372982609171
Step is: 40800 | Cost is: 0.385623392745
Step is: 40850 | Cost is: 0.348453052496
Step is: 40900 | Cost is: 0.409424094216
Step is: 40950 | Cost is: 0.377065274144
Step is: 41000 | Cost is: 0.337431055324
Step is: 41050 | Cost is: 0.338940980371
Step is: 41100 | Cost is: 0.351764650303
Step is: 41150 | Cost is: 0.373207830187
Step is: 41200 | Cost is: 0.398673945959
Step is: 41250 | Cost is: 0.391300687374
Step is: 41300 | Cost is: 0.378846200065
Step is: 41350 | Cost is: 0.351729307407
Step is: 41400 | Cost is: 0.348799147532
Step is: 41450 | Cost is: 0.338082484938
Step is: 41500 | Cost is: 0.402889457447
Step is: 41550 | Cost is: 0.407392852109
Step is: 41600 | Cost is: 0.317492067237
Step is: 41650 | Cost is: 0.351357892377
Step is: 41700 | Cost is: 0.376444524691
Step is: 41750 | Cost is: 0.37851013883
Step is: 41800 | Cost is: 0.368247597379
Step is: 41850 | C

Step is: 50700 | Cost is: 0.34651202334
Step is: 50750 | Cost is: 0.370075630942
Step is: 50800 | Cost is: 0.34588699275
Step is: 50850 | Cost is: 0.365170621977
Step is: 50900 | Cost is: 0.370401743531
Step is: 50950 | Cost is: 0.338484002376
Step is: 51000 | Cost is: 0.350021571271
Step is: 51050 | Cost is: 0.365766568039
Step is: 51100 | Cost is: 0.375621724492
Step is: 51150 | Cost is: 0.364096269379
Step is: 51200 | Cost is: 0.381927356696
Step is: 51250 | Cost is: 0.402909106371
Step is: 51300 | Cost is: 0.367650406631
Step is: 51350 | Cost is: 0.39319556171
Step is: 51400 | Cost is: 0.349679909771
Step is: 51450 | Cost is: 0.330327617683
Step is: 51500 | Cost is: 0.291160691921
Step is: 51550 | Cost is: 0.416435948067
Step is: 51600 | Cost is: 0.355776759664
Step is: 51650 | Cost is: 0.382036052803
Step is: 51700 | Cost is: 0.308954185111
Step is: 51750 | Cost is: 0.330017793629
Step is: 51800 | Cost is: 0.383007829055
Step is: 51850 | Cost is: 0.364294838361
Step is: 51900 | Co

Step is: 60750 | Cost is: 0.356564169248
Step is: 60800 | Cost is: 0.363941423812
Step is: 60850 | Cost is: 0.386670629172
Step is: 60900 | Cost is: 0.341845168244
Step is: 60950 | Cost is: 0.334470554537
Step is: 61000 | Cost is: 0.385327144338
Step is: 61050 | Cost is: 0.378553415708
Step is: 61100 | Cost is: 0.379118335356
Step is: 61150 | Cost is: 0.411428909883
Step is: 61200 | Cost is: 0.373208431986
Step is: 61250 | Cost is: 0.363242764055
Step is: 61300 | Cost is: 0.390726237769
Step is: 61350 | Cost is: 0.416185881875
Step is: 61400 | Cost is: 0.375083903194
Step is: 61450 | Cost is: 0.363255801843
Step is: 61500 | Cost is: 0.363380040409
Step is: 61550 | Cost is: 0.331805008509
Step is: 61600 | Cost is: 0.365907624514
Step is: 61650 | Cost is: 0.37825219673
Step is: 61700 | Cost is: 0.372430166703
Step is: 61750 | Cost is: 0.384375850607
Step is: 61800 | Cost is: 0.389343615551
Step is: 61850 | Cost is: 0.361631340361
Step is: 61900 | Cost is: 0.404165905692
Step is: 61950 | 

KeyboardInterrupt: 

## Part 2

In [ ]:
layer_dimensions = [3072,..,.., 10]  # including the input and output layers  
# 3072 is the input feature size, 10 is the number of outputs in the final layer
FCN = FullyConnectedNetwork(layer_dimensions)
FCN.train(X_train, Y_train, max_iters=10000, batch_size=200, learning_rate=0.01, lambd=0.1,validate_every=200)
# lambd, the L2 regularization penalty hyperparamter will not be 0 for this part
y_predicted = FCN.evaluate(X_test)  # print accuracy on test set

## Extra Credit